## Generate synthetic data
We can utilize one more powerful data to help us generate sft dataset.

In [1]:
from openai import OpenAI
from key import API_KEY
client = OpenAI(api_key=API_KEY)

In [2]:
instruction = """\
你是一個非常聰明的助手，現在你的使用者想要運用給定的檔案內的資料，生成一份問答題的訓練資料。\
使用者會給你一些範例，請你參考這些範例，生成問答資料集。題目的多樣性最好能越多越好，與使用者的格式不相同也沒有關係。\
重點是必須與給定的資料有關係。
"""

In [3]:
import re
with open("道路交通安全規則.txt", "r") as f:
    text = f.read()

sections = text.split("## ")

In [28]:
def generate_prompt(num_request: int = 4, section_idx: int = -1):
    import random
    if section_idx == -1:
        section_idx = random.randint(0, len(sections)-1)
    data = sections[section_idx]
    
    # Dyanmic number of requests
    num_request = len(data) // 100 + 1

    example = """\
{\"question\": \"車輛申請過戶需不需要申請易動登記？\", \"answer\": \"需要\"}
{\"question\": \"汽車定期檢驗中座位需與什麼核定數相符？\", \"answer\": \"需與行車執照登載核定數相符\"}
"""

    request = """\
"以下是道路交通安全規則的部份內容:
```
%s
```
請依據以上內容，生成%d筆類似以下範例的題目。請直接輸出生成的題目即可。不需輸出編號或其他資訊。
範例：
%s
""" % (data, num_request, example)
    
    print("\033[92m" + request + "\033[0m")

    return request

In [29]:
def get_response(request):
  return client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": instruction
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": request
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    response_format={
      "type": "text"
    }
  )

### Preview
After the response is generated, we can check the quality of the generated data here.

In [33]:
for i in range(30):
    request = generate_prompt()
    response = get_response(request)
    data = response.choices[0].message.content
    print(data)
    import json
    try:
        data = [json.loads(l) for l in data.split("\n") if l]
        with open("data/train.jsonl", "a") as f:
            for l in data:
                f.write(json.dumps(l, ensure_ascii=False) + "\n")
    except json.JSONDecodeError:
        print("Generated data is not in correct format.")
        print(data)

"以下是道路交通安全規則的部份內容:
```
第39條
汽車牌照登記書或過戶登記書如有遺失，應申請補發。


```
請依據以上內容，生成1筆類似以下範例的題目。請直接輸出生成的題目即可。不需輸出編號或其他資訊。
範例：
{"question": "車輛申請過戶需不需要申請易動登記？", "answer": "需要"}
{"question": "汽車定期檢驗中座位需與什麼核定數相符？", "answer": "需與行車執照登載核定數相符"}


{"question": "如果汽車的牌照登記書遺失了，應該怎麼辦？", "answer": "應申請補發"}
"以下是道路交通安全規則的部份內容:
```
第46條
機車申請牌照檢驗項目及基準如下：
一、引擎或車身號碼與來歷憑證相符。
二、前後煞車效能合於規定。
三、前後輪左右偏差合於規定。
四、各種喇叭合於規定且不得裝設可發出不同音調之喇叭。
五、各種燈光與標誌應符合附件七規定。
六、車輛型式、顏色與紀錄相符。
七、左右兩側之照後鏡、擋泥板合於規定。
八、各部機件齊全作用正常。
九、不得加掛邊車。
十、小型輕型機車車輛空重（含電池）應在七十公斤以下。
十一、小型輕型機車之輪胎直徑應在三百公釐以上，四百二十公釐以下，輪胎寬度應在七十五公釐以上，一百公釐以下。
十二、小型輕型機車之超速斷電功能應合於車速超過每小時四十五公里，電動機電源應能於三秒內自動暫停供電之規定。小型輕型機車之故障斷電功能應合於控制系統超速訊號輸入線短路或斷路，三秒內電動機電源應能自動斷電之規定。
十三、車輛尺度應合於第三十八條規定。
十四、輪胎之胎面未磨損至中華民國國家標準CNS4879機車用輪胎標準所定之任一胎面磨耗指示平臺。
大型重型機車定期檢驗之項目及基準，依前項申請牌照檢驗規定辦理。
古董車申請機車專用牌照檢驗或定期檢驗之項目及基準，依第一項申請牌照檢驗規定辦理。但車輛尺度、重量為原廠規格，或原廠設計無引擎或車身（架）號碼、照後鏡、輪胎、喇叭、擋泥板及各項燈光等設備，得免合於各該規定。


```
請依據以上內容，生成6筆類似以下範例的題目。請直接輸出生成的題目即可。不需輸出編號或其他資訊。
範例：
{"question": "車輛申請過戶需不需要申請易動登記？", "answer": "需要"}
{"question": "汽車定期檢驗中座位